<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
%pylab nbagg
from astropy.io import fits
from astropy.table import Table

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
ifile = 'Results/Reticulum_config2_epoch1_SCIENCE_RBNSPECTRA.fits' # input file
odir = 'flames_1dspec/' # output directory

# this loads the fits file
hdu = fits.open(ifile)

# here we see the structure
hdu.info()

In [ ]:
# read table
t = Table.read(ifile)

# wavelenght
ext=0 
crval = hdu[ext].header['crval2']
cdelt = hdu[ext].header['cdelt2']
naxis = hdu[ext].header['naxis2']

wl = crval+cdelt*np.arange(naxis)

#other relevant keywords
cunit = hdu[ext].header['cunit2']
mjd = hdu[ext].header['MJD-OBS']
bunit = hdu[ext].header['bunit']
exptime = hdu[ext].header['exptime']

# iterate over all fiber spectra
for i in range(hdu[3].data.shape[1]):
    
    #create a new hdu object with sky sub. spec. of fiber i
    _hdu = fits.PrimaryHDU(hdu[3].data[:,i])
    
    #add header info
    _hdu.header['crval1'] = crval
    _hdu.header['cdelt1'] = cdelt
    _hdu.header['cunit1'] = cunit
    
    _hdu.header['bunit'] = bunit
    _hdu.header['object'] = t['OBJECT'][i]
    _hdu.header['ra'] = t['RA'][i]
    _hdu.header['dec'] = t['DEC'][i]
    _hdu.header['mag'] = np.round(t['MAGNITUDE'][i],3)
    _hdu.header['MJD-OBS'] = mjd
    _hdu.header['EXPTIME'] = exptime 
    
    header = _hdu.header
    header.comments['crval1'] = 'value of ref pixel'
    header.comments['cdelt1'] = 'Binning factor'
    header.comments['naxis1'] = 'length of data axis 1'
    header.comments['exptime'] = 'Total integration time'
    header.comments['mjd-obs'] = 'MJD start'
    
    #write file
    ofile = '{:}.fits'.format(t['OBJECT'][i].strip())
    print('writing: {:}'.format(odir+ofile))
    _hdu.writeto(odir+ofile,overwrite=True)
    
#     break